# Introduction

This is a proyect with real data to ratemake a product in DCI BRASIl

# Data preparation

- First of all we did an extraction of the policy and claims dataset (8/27/2024) and stores it in a parquet format. This will be the base file for our development.
- The cut date is July 31

In [1]:
# for our data preparation we will focus on the polars package
import polars as pl
from datetime import date
import hvplot.polars

- Our policy dataframe has **1_480_500** entries. Each one is policy-location-coverage level. Initially we have 35 columns. It is stores in the `raw_policy_data`

- Our claims dataframe has **81_101** entries. It is stored in `raw_claims` variable.

In [2]:
# Opening the raw policy-file
path = r"C:\Users\abguerr\OneDrive - Chubb\2024\Ratemaking-Brazil\Archivo\parquet_data"
name = "policy_info_27-8-2024"
raw_policy_data = pl.read_parquet(source=path + f"\\{name}.parquet")

# Opening the raw claims-file
name_claims = "claims_info_9-8-2024"
raw_claims = pl.read_parquet(source=path + f"\\{name_claims}.parquet")

# Opening score serasa
name_score_serasa = "score_serasa"
score_serasa = pl.read_parquet(source=path + f"\\{name_score_serasa}.parquet")


# auxiliar functions
def show_polars(
    df, all_columns=True, all_rows=False, number_of_rows=None, width_chars=1000
):
    """This function allows to print polars dataframe with a cool style

    Args:
        df (_type_): _description_
        all_columns (bool, optional): _description_. Defaults to True.
        all_rows (bool, optional): _description_. Defaults to False.
        number_of_rows (_type_, optional): _description_. Defaults to None.
    """
    if all_columns:
        tbl_cols = -1
    else:
        tbl_cols = None
    if all_rows:
        tbl_rows = -1
    elif number_of_rows != None:
        tbl_rows = number_of_rows
    else:
        tbl_rows = None

    with pl.Config(
        set_fmt_float="full",
        thousands_separator=",",
        decimal_separator=".",
        float_precision=2,
        tbl_rows=tbl_rows,
        tbl_cols=tbl_cols,
        tbl_width_chars=width_chars,
    ):

        print(df)


def summary(df, width_chars=1000):

    with pl.Config(
        set_fmt_float="full",
        thousands_separator=",",
        decimal_separator=".",
        float_precision=3,
        tbl_rows=-1,
        tbl_width_chars=width_chars,
    ):
        empty = (
            df.fill_nan(None)
            .describe()
            .filter(pl.col("statistic").is_in(["count", "null_count", "min", "max"]))
            .transpose(
                include_header=True, column_names=["count", "null_count", "min", "max"]
            )
        ).filter(pl.col("column") != "statistic")
        print(empty)


def value_counts_df(df, column_list):
    with pl.Config(
        set_fmt_float="full",
        thousands_separator=",",
        decimal_separator=".",
        float_precision=3,
        tbl_rows=-1,
    ):
        for i in column_list:
            with pl.Config(tbl_rows=-1):
                h = (
                    df.get_column(i)
                    .value_counts(normalize=True)
                    .sort(by="proportion", descending=True)
                )
                print(h)


# showing sample of a policy
show_polars(raw_policy_data)
print("*" * 500)
show_polars(raw_claims)

shape: (1_480_500, 33)
┌───────────────┬─────────────┬─────────────────────────────────┬───────────┬─────────────────┬──────────────┬─────────────────────────────────┬──────────────┬─────────────────┬─────────┬────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬───────────────────────┬─────────────────────────────────┬─────────────────────────────────┬───────────────────────┬──────────────┐
│ policy_number ┆ location_id ┆ cobertura                       ┆ coverage  ┆ inicio_vigencia ┆ fin_vigencia ┆ REGION                          ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY               ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM200 ┆

## Policy example 1
`policy_number = 1_160_012_102` This is single location.

In [3]:
show_polars(
    raw_policy_data.filter(pl.col("policy_number") == 1_160_012_102), all_rows=True
)
print("*" * 300)

show_polars(raw_claims.filter(pl.col("policy_number") == 1_160_012_102), all_rows=True)

shape: (10, 33)
┌───────────────┬─────────────┬─────────────────────────────────┬───────────────────┬─────────────────┬──────────────┬────────────────────────────┬──────────────┬────────────────┬─────────┬──────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬─────────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────────────────┬──────────────┐
│ policy_number ┆ location_id ┆ cobertura                       ┆ coverage          ┆ inicio_vigencia ┆ fin_vigencia ┆ REGION                     ┆ TIPO_EMISION ┆ PRODUCT_NAME   ┆ RUBRICA ┆ INDUSTRY         ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM200 ┆ Brigada_de_Incendio ┆

## Policy Example 2

`policy_number =  1_180_072_313` This is multilocation.

In [4]:
show_polars(
    raw_policy_data.filter(pl.col("policy_number") == 1_180_072_313), all_rows=True
)
print("*" * 300)

show_polars(raw_claims.filter(pl.col("policy_number") == 1_180_072_313), all_rows=True)

shape: (555, 33)
┌───────────────┬─────────────┬──────────────────────────────┬───────────────────┬─────────────────┬──────────────┬─────────────────────────────────┬──────────────┬─────────────────┬─────────┬───────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬───────────────┬────────────┬────────────┬───────────────────────┬─────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┐
│ policy_number ┆ location_id ┆ cobertura                    ┆ coverage          ┆ inicio_vigencia ┆ fin_vigencia ┆ REGION                          ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY      ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM200 ┆ Brigada_de_Incendio ┆ Alar

## Duplicated entries into policy table

- We will use the key policy_number + location_id + cobertura * product_name
- We found 243 entries repeated under this key. Further investigation is needed to identify why they are duplicated.

In [5]:
# key columns
policy_num = pl.col("policy_number")
location_id = pl.col("location_id")
cobertura = pl.col("cobertura")
product_name = pl.col("PRODUCT_NAME")

# duplicated of the dataframe

show_polars(raw_policy_data.filter(raw_policy_data.is_duplicated()))

# lets identy initially the duplicated rows by the key
unique_key = raw_policy_data.select(
    unique_key=pl.concat_str(
        [policy_num, location_id, cobertura, product_name], separator="-"
    )
).is_duplicated()

# filtering by duplicated
show_polars(raw_policy_data.filter(unique_key), number_of_rows=50)

# dropping duplicated values out
raw_policy_data = raw_policy_data.unique(
    subset=["policy_number", "location_id", "cobertura", "PRODUCT_NAME"]
)
print(f"The new values of the policy dataframe are {raw_policy_data.shape}")

shape: (264, 33)
┌───────────────┬─────────────┬─────────────────┬───────────────────┬─────────────────┬──────────────┬─────────────────────────────────┬──────────────┬─────────────────┬─────────┬───────────────────┬────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬─────────────┬─────────────────┬─────────────────────────────────┬───────────────────────┬──────────────┐
│ policy_number ┆ location_id ┆ cobertura       ┆ coverage          ┆ inicio_vigencia ┆ fin_vigencia ┆ REGION                          ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY          ┆ OCCUPANCE                      ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM200 ┆ Brigada_de_Incendio ┆ Alarme_Local ┆ Alarme_Monitorado_24_H

## Casting the values in the exposure dataset

- Dates comes in string type, changed to date.
- Decimal columns changed to simple floating values
- Binary variables changed to int 8 to reduces storage

In [6]:
policy_binary = [
    "Extintores",
    "Hidrantes",
    "Sprinklers",
    "Detectores_de_Fumaça_ou_Calor",
    "CO2_ou_FM200",
    "Brigada_de_Incendio",
    "Alarme_Local",
    "Alarme_Monitorado_24_Horas",
    "Sistema_de_Filmagem",
    "Vigilancia",
    "Cofre",
]

# previous datatypes
print(raw_policy_data.dtypes)

# we will update our dataframe to cast the date columns to the proper data structure
raw_policy_data = raw_policy_data.with_columns(
    pl.col(["inicio_vigencia", "fin_vigencia"]).str.to_date()
)
# type for the TIV
raw_policy_data = raw_policy_data.with_columns(
    pl.col("total_insured_value").cast(pl.Float64),
    pl.col(policy_binary).cast(pl.Int8),
)

# new datatypes
print(raw_policy_data.dtypes)

[Int64, Int64, String, String, String, String, String, String, String, String, String, String, String, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, String, Int64, Float64, String, String, String, Decimal(precision=38, scale=13), Float64]
[Int64, Int64, String, String, Date, Date, String, String, String, String, String, String, String, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int64, String, Int64, Float64, String, String, String, Float64, Float64]


## Cleaning empty values

- For categorical variables we introduced the "unknown" category.
- For Binary variable unknown is fixed to zero

In [7]:
summary(raw_policy_data, width_chars=2000)

fill_null_with_unknown = [
    "RUBRICA",
    "INDUSTRY",
    "OCCUPANCE",
    "MACRO_REGION",
    "TABELA",
    "NAICS_GROUP",
    "NAICS_OLD_GROUP",
    "NAICS_DESCRIPTION",
]

# cleaning the region field
region = pl.col("REGION")
raw_policy_data = (
    raw_policy_data.lazy()
    .with_columns(region.fill_null("unknown"))
    .with_columns(
        pl.when(region == "NOT INFORMED")
        .then(pl.lit("unknown"))
        .otherwise(region)
        .alias("region")
    )
    .drop(["REGION"])
    .with_columns(
        pl.col(fill_null_with_unknown).fill_null("unknown"),
        pl.col("CD_NAICS").fill_null(0),
    )
    .with_columns(pl.col(policy_binary).fill_nan(None).fill_null(pl.lit(0)))
    .collect()
)
summary(raw_policy_data, width_chars=2000)

shape: (33, 5)
┌───────────────────────────────┬───────────┬────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ column                        ┆ count     ┆ null_count ┆ min                             ┆ max                             │
│ ---                           ┆ ---       ┆ ---        ┆ ---                             ┆ ---                             │
│ str                           ┆ str       ┆ str        ┆ str                             ┆ str                             │
╞═══════════════════════════════╪═══════════╪════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ policy_number                 ┆ 1480257.0 ┆ 0.0        ┆ 1160007690.0                    ┆ 1990023208.0                    │
│ location_id                   ┆ 1480257.0 ┆ 0.0        ┆ 1.0                             ┆ 25822368.0                      │
│ cobertura                     ┆ 1480257   ┆ 0          ┆ Alagamento                      ┆ Ven

## Adding Score serasa column (Under construction)

In [ ]:
show_polars(score_serasa)
score_serasa = (
    score_serasa.with_columns(pl.col("policy_number").cast(pl.Int64))
    .drop(["year"])
    .unique(subset=["policy_number", "score"])
)

print(score_serasa.n_unique(["policy_number", "score"]))
show_polars(score_serasa)
raw_policy_data.join(score_serasa, on="policy_number", how="left").with_columns(
    pl.col("score").fill_null(0)
)

## Cleaning Claim Dataframe

- We drop entries with empty policy number
- we lost one entry parsing dt_event to the proper 
- Empty cobertura and coverage changed to the category "unknown"

## Raw reported value

- Since 2019 we had approximately 240 M reals incurred. (43 M USD)

In [8]:
show_polars(raw_claims.sum())

shape: (1, 7)
┌────────────────────┬───────────────────┬───────────┬──────────┬──────────────┬───────────────────┬────────────────┐
│ policy_number      ┆ location_id       ┆ cobertura ┆ coverage ┆ claim_number ┆ dt_event          ┆ il_orig        │
│ ---                ┆ ---               ┆ ---       ┆ ---      ┆ ---          ┆ ---               ┆ ---            │
│ i64                ┆ i64               ┆ str       ┆ str      ┆ str          ┆ i64               ┆ f64            │
╞════════════════════╪═══════════════════╪═══════════╪══════════╪══════════════╪═══════════════════╪════════════════╡
│ 97,084,860,339,169 ┆ 1,827,149,435,900 ┆ null      ┆ null     ┆ null         ┆ 1,639,283,219,801 ┆ 240,729,445.42 │
└────────────────────┴───────────────────┴───────────┴──────────┴──────────────┴───────────────────┴────────────────┘


In [9]:
## Cleaning Claim dataframe

raw_claims = raw_claims.fill_nan(None)  # nan != null

# Fixing date occurrence to the proper format
raw_claims = raw_claims.with_columns(pl.col("dt_event").cast(pl.String)).with_columns(
    pl.col("dt_event").str.to_date(format="%Y%m%d", strict=False)
)

# Counting null values
print("Null values")
show_polars(raw_claims.null_count())

# showing entries with null policy_number
print("\nEmpty Policy number \n")
show_polars(raw_claims.filter(policy_num.is_null()))

# reported value of empty entries
print("total reported value for empty policy number")
show_polars(raw_claims.filter(policy_num.is_null()).select(pl.col("il_orig").sum()))

# showing entries with empty cobertura
print("\nEmpty cobertura \n")
show_polars(raw_claims.filter(cobertura.is_null()))

# reported value of empty cobertura
print("total reported value for empty policy number")
show_polars(raw_claims.filter(cobertura.is_null()).select(pl.col("il_orig").sum()))

# cobertura and coverage empty values changed to "unknown"
raw_claims = raw_claims.fill_null("unknown")

# keeping only entries with policy_number
raw_claims = raw_claims.filter(pl.col("policy_number").is_not_null())

# new empties
show_polars(raw_claims.null_count())

Null values
shape: (1, 7)
┌───────────────┬─────────────┬───────────┬──────────┬──────────────┬──────────┬─────────┐
│ policy_number ┆ location_id ┆ cobertura ┆ coverage ┆ claim_number ┆ dt_event ┆ il_orig │
│ ---           ┆ ---         ┆ ---       ┆ ---      ┆ ---          ┆ ---      ┆ ---     │
│ u32           ┆ u32         ┆ u32       ┆ u32      ┆ u32          ┆ u32      ┆ u32     │
╞═══════════════╪═════════════╪═══════════╪══════════╪══════════════╪══════════╪═════════╡
│ 50            ┆ 8,492       ┆ 534       ┆ 534      ┆ 0            ┆ 1        ┆ 0       │
└───────────────┴─────────────┴───────────┴──────────┴──────────────┴──────────┴─────────┘

Empty Policy number 

shape: (50, 7)
┌───────────────┬─────────────┬───────────────────────┬───────────────────┬───────────────┬────────────┬───────────┐
│ policy_number ┆ location_id ┆ cobertura             ┆ coverage          ┆ claim_number  ┆ dt_event   ┆ il_orig   │
│ ---           ┆ ---         ┆ ---                   ┆ ---      

## Dealing with Honorários de peritos

- Inside the claims dataset we have some coverage corresponding to ALAE. We analyse them and put it apart.

In [10]:
# taking out Honorários de Peritos

honorarios = raw_claims.filter(
    (pl.col("cobertura") == "Honorários de Peritos")
    | (pl.col("cobertura") == "Pagamento de Honorarios")
    | (pl.col("cobertura") == "Despesas de Cosseguro")
    | (pl.col("cobertura") == "Despesas de Salvados")
    | (pl.col("cobertura") == "Despesas de Ressarcimento")
    | (pl.col("cobertura") == "Honorários de Cosseguro")
)
show_polars(honorarios)
show_polars(honorarios.sum())

# filtering
pure_claims = raw_claims.filter(
    (pl.col("cobertura") != "Honorários de Peritos")
    & (pl.col("cobertura") != "Pagamento de Honorarios")
    & (pl.col("cobertura") != "Despesas de Cosseguro")
    & (pl.col("cobertura") != "Despesas de Salvados")
    & (pl.col("cobertura") != "Despesas de Ressarcimento")
    & (pl.col("cobertura") != "Honorários de Cosseguro")
    & (pl.col("cobertura") != "unknown")
)

#
show_polars(pure_claims)

shape: (32_232, 7)
┌───────────────┬─────────────┬─────────────────────────┬──────────┬───────────────┬────────────┬─────────┐
│ policy_number ┆ location_id ┆ cobertura               ┆ coverage ┆ claim_number  ┆ dt_event   ┆ il_orig │
│ ---           ┆ ---         ┆ ---                     ┆ ---      ┆ ---           ┆ ---        ┆ ---     │
│ i64           ┆ i64         ┆ str                     ┆ str      ┆ str           ┆ date       ┆ f64     │
╞═══════════════╪═════════════╪═════════════════════════╪══════════╪═══════════════╪════════════╪═════════╡
│ 1,160,005,853 ┆ null        ┆ Pagamento de Honorarios ┆ OTHER    ┆ 0201619001780 ┆ 2019-04-25 ┆ 850.00  │
│ 1,160,006,564 ┆ null        ┆ Pagamento de Honorarios ┆ OTHER    ┆ 0405119000410 ┆ 2019-01-31 ┆ 0.00    │
│ 1,160,014,779 ┆ 24,841,919  ┆ Pagamento de Honorarios ┆ OTHER    ┆ 0301620000440 ┆ 2019-12-13 ┆ 0.00    │
│ 1,160,007,920 ┆ null        ┆ Pagamento de Honorarios ┆ OTHER    ┆ 0201619002190 ┆ 2019-05-30 ┆ 4.21    │
│ 1,180,0

## Finding large claims

In [11]:
flag_large = (
    pure_claims.group_by(["claim_number"])
    .agg(
        pl.col("il_orig").sum().alias("total_il"),
        pl.col("cobertura").n_unique().alias("num_coverages"),
    )
    .sort(by=["num_coverages", "total_il"], descending=True)
    .with_columns(
        pl.when(pl.col("total_il") > 550_194)
        .then(pl.lit("large"))
        .otherwise(pl.lit("attritional"))
        .alias("flag_large")
    )
)
show_polars(flag_large, number_of_rows=50)

show_polars(
    flag_large.group_by("flag_large")
    .agg(pl.col("total_il").sum(), pl.col("claim_number").n_unique())
    .with_columns(total_il_percentage=pl.col("total_il") / pl.col("total_il").sum())
)

shape: (20_438, 4)
┌───────────────┬──────────────┬───────────────┬─────────────┐
│ claim_number  ┆ total_il     ┆ num_coverages ┆ flag_large  │
│ ---           ┆ ---          ┆ ---           ┆ ---         │
│ str           ┆ f64          ┆ u32           ┆ str         │
╞═══════════════╪══════════════╪═══════════════╪═════════════╡
│ 0201822005370 ┆ 6,681,278.82 ┆ 1             ┆ large       │
│ 0201824001020 ┆ 6,200,000.00 ┆ 1             ┆ large       │
│ 0201823004120 ┆ 4,630,000.00 ┆ 1             ┆ large       │
│ 0201822005400 ┆ 2,721,712.15 ┆ 1             ┆ large       │
│ 0301824000010 ┆ 2,160,000.00 ┆ 1             ┆ large       │
│ 0501822000590 ┆ 1,664,853.23 ┆ 1             ┆ large       │
│ 0901824000240 ┆ 1,604,000.00 ┆ 1             ┆ large       │
│ 0701820000150 ┆ 1,423,877.36 ┆ 1             ┆ large       │
│ 0201819002210 ┆ 1,362,948.12 ┆ 1             ┆ large       │
│ 0201819001940 ┆ 1,343,403.69 ┆ 1             ┆ large       │
│ 0201819001520 ┆ 1,289,564.18 ┆ 1  

## Grouping claims dataset

In [12]:
# Getting the factors to transform values to ultimate. and cat claims

archivopath = r"C:\Users\abguerr\OneDrive - Chubb\2024\Ratemaking-Brazil\Archivo"

# Development factors
to_ultimate = pl.read_excel(
    source=archivopath + f"\\excel_files\\Factores_Brazil.xlsx",
    sheet_name="to_ultimate",
    schema_overrides={"key": pl.String, "year-month": pl.String},
)
show_polars(to_ultimate)

# CAT claims
cat_claims = pl.read_excel(
    source=archivopath + f"\\excel_files\\BASE_CAT.xlsx",
    sheet_name="Plan1",
    schema_overrides={"claim_number": pl.String, "CAT-flag": pl.String},
)
show_polars(cat_claims)

# adding the year-month variable to our dataset
raw_claims = raw_claims.with_columns(
    pl.concat_str(
        [
            pl.col("dt_event").dt.year(),
            pl.col("dt_event").dt.strftime("%m"),
        ]
    ).alias("year-month")
    # we mark GL coverage vs NoneGL coverage
)

# grouping by the key policy_number + cobertura + coverage + claim_number + year-month
# adding the flag large

grouped_claims = (
    raw_claims.group_by(
        pl.col("policy_number", "claim_number", "cobertura", "coverage", "year-month")
    ).agg(pl.col("il_orig").sum())
    # flag large or attritional
    .join(flag_large, on="claim_number", how="left")
)
show_polars(grouped_claims)


grouped_claims = (
    # adding key column
    grouped_claims.with_columns(
        pl.when(pl.col("coverage") == "GL")
        .then(pl.col("year-month") + pl.lit("GL"))
        .otherwise(pl.col("year-month") + pl.lit("NONGL"))
        .alias("key")
    )
    # bringing to ultimate
    .join(to_ultimate, on="key", how="left")
    # adding CAT FLAG
    .join(cat_claims, on="claim_number", how="left").with_columns(
        pl.col("CAT-flag").fill_null("NON_CAT")
    )
    # bringing to ultimate
    .with_columns(
        ultimate_il_orig=pl.when(
            (pl.col("flag_large") == "large") | (pl.col("CAT-flag") == "CAT")
        )
        .then(pl.col("il_orig") * pl.col("inflacion"))
        .otherwise(pl.col("il_orig") * pl.col("IL_DFM") * pl.col("inflacion"))
    )
    # adding CAT FLAG
    # .join(cat_claims, on="claim_number", how="left")
    # .with_columns(pl.col("CAT-flag").fill_null("NON_CAT"))
)

show_polars(grouped_claims)
show_polars(grouped_claims.null_count())

# Merging the dataframes
# raw_claims = raw_claims.join(to_ultimate, on="key", how="left")
# show_polars(raw_claims)
# show_polars(raw_claims.null_count())

shape: (134, 6)
┌─────────────┬────────────┬───────┬────────┬────────┬───────────┐
│ key         ┆ year-month ┆ line  ┆ IL_DFM ┆ CC_DFM ┆ inflacion │
│ ---         ┆ ---        ┆ ---   ┆ ---    ┆ ---    ┆ ---       │
│ str         ┆ str        ┆ str   ┆ f64    ┆ f64    ┆ f64       │
╞═════════════╪════════════╪═══════╪════════╪════════╪═══════════╡
│ 202407NONGL ┆ 202407     ┆ NONGL ┆ 3.40   ┆ 3.56   ┆ 1.00      │
│ 202407GL    ┆ 202407     ┆ GL    ┆ 4.13   ┆ 3.99   ┆ 1.00      │
│ 202406NONGL ┆ 202406     ┆ NONGL ┆ 1.70   ┆ 1.78   ┆ 1.00      │
│ 202406GL    ┆ 202406     ┆ GL    ┆ 2.06   ┆ 1.99   ┆ 1.00      │
│ 202405NONGL ┆ 202405     ┆ NONGL ┆ 1.22   ┆ 1.27   ┆ 1.00      │
│ …           ┆ …          ┆ …     ┆ …      ┆ …      ┆ …         │
│ 201903GL    ┆ 201903     ┆ GL    ┆ 1.00   ┆ 1.00   ┆ 1.31      │
│ 201902NONGL ┆ 201902     ┆ NONGL ┆ 1.00   ┆ 1.00   ┆ 1.32      │
│ 201902GL    ┆ 201902     ┆ GL    ┆ 1.00   ┆ 1.00   ┆ 1.32      │
│ 201901NONGL ┆ 201901     ┆ NONGL ┆ 1.00   ┆ 

## Incurred values per year-month

In [13]:
h = (
    grouped_claims.group_by(pl.col("year-month"))
    .agg(
        pl.col("claim_number").n_unique().alias("claim count"),
        pl.col("ultimate_il_orig").sum(),
        pl.col("policy_number").n_unique().alias("num_accounts"),
        pl.col("flag_large").value_counts(),
        pl.col("CAT-flag").value_counts(),
    )
    .sort(by="year-month")
    .with_columns(
        claim_percentage=pl.col("claim count") / pl.col("claim count").sum() * 100,
        ultimate_percentage=pl.col("ultimate_il_orig")
        / pl.col("ultimate_il_orig").sum()
        * 100,
    )
)


show_polars(h, number_of_rows=68)

print("Without large and CAT")

# final claims

final_claims = grouped_claims.filter(
    (pl.col("flag_large") != "large") & (pl.col("CAT-flag") != "CAT")
)

# cleaning unnecesary columns
final_claims = (
    final_claims.with_columns(date=pl.col("year-month").str.to_datetime(format="%Y%m"))
    .with_columns(
        year=pl.col("date").dt.year(),
        month=pl.col("date").dt.month(),
        quarter=pl.col("date").dt.quarter(),
    )
    .drop(
        [
            "date",
            "year-month",
            "year-month_right",
            "key",
            "il_orig",
            "total_il",
            "num_coverages",
            "inflacion",
            "IL_DFM",
            "flag_large",
            "CAT-flag",
        ]
    )
)

show_polars(final_claims)

# show_polars(h.filter())

shape: (68, 8)
┌────────────┬─────────────┬──────────────────┬──────────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────────┬─────────────────────┐
│ year-month ┆ claim count ┆ ultimate_il_orig ┆ num_accounts ┆ flag_large                      ┆ CAT-flag                        ┆ claim_percentage ┆ ultimate_percentage │
│ ---        ┆ ---         ┆ ---              ┆ ---          ┆ ---                             ┆ ---                             ┆ ---              ┆ ---                 │
│ str        ┆ u32         ┆ f64              ┆ u32          ┆ list[struct[2]]                 ┆ list[struct[2]]                 ┆ f64              ┆ f64                 │
╞════════════╪═════════════╪══════════════════╪══════════════╪═════════════════════════════════╪═════════════════════════════════╪══════════════════╪═════════════════════╡
│ null       ┆ 1           ┆ 0.00             ┆ 1            ┆ [{"attritional",1}]             ┆ [{"NON_CAT",1}]             

## Reported events per year month

In [15]:
claim_count = (
    final_claims.group_by(pl.col("year", "month", "quarter", "line"))
    .agg(pl.col("claim_number").n_unique())
    .with_columns(
        pl.concat_str([pl.col("year") * 100 + pl.col("month"), "line"]).alias("key")
    )
    .join(to_ultimate, on="key", how="left")
    .drop(["key", "year-month", "line_right", "IL_DFM", "inflacion"])
    # ultimate claim_number
    .with_columns(ultimate_claim_number=pl.col("claim_number") * pl.col("CC_DFM"))
    .drop(["CC_DFM"])
)

# claim_count_quarter =


show_polars(claim_count, number_of_rows=50)

shape: (135, 6)
┌───────┬───────┬─────────┬───────┬──────────────┬───────────────────────┐
│ year  ┆ month ┆ quarter ┆ line  ┆ claim_number ┆ ultimate_claim_number │
│ ---   ┆ ---   ┆ ---     ┆ ---   ┆ ---          ┆ ---                   │
│ i32   ┆ i8    ┆ i8      ┆ str   ┆ u32          ┆ f64                   │
╞═══════╪═══════╪═════════╪═══════╪══════════════╪═══════════════════════╡
│ 2,024 ┆ 4     ┆ 2       ┆ NONGL ┆ 194          ┆ 213.10                │
│ 2,024 ┆ 2     ┆ 1       ┆ GL    ┆ 61           ┆ 66.54                 │
│ 2,024 ┆ 6     ┆ 2       ┆ GL    ┆ 38           ┆ 75.74                 │
│ 2,023 ┆ 8     ┆ 3       ┆ GL    ┆ 75           ┆ 76.96                 │
│ 2,020 ┆ 10    ┆ 4       ┆ GL    ┆ 72           ┆ 72.00                 │
│ 2,022 ┆ 1     ┆ 1       ┆ GL    ┆ 76           ┆ 76.11                 │
│ 2,022 ┆ 10    ┆ 4       ┆ GL    ┆ 59           ┆ 59.39                 │
│ 2,020 ┆ 7     ┆ 3       ┆ GL    ┆ 62           ┆ 62.00                 │
│ 2,024 ┆

## Example for a couple of policies

In [16]:
# lets see our examples 1_160_012_102 and  1_180_072_313
for i in [1_160_012_102, 1_180_072_313]:
    show_polars(raw_claims.filter(pl.col("policy_number") == i))
    show_polars(final_claims.filter(pl.col("policy_number") == i))

print("*" * 50)
show_polars(final_claims)
show_polars(final_claims.null_count())

shape: (6, 8)
┌───────────────┬─────────────┬─────────────────────────────────┬──────────────┬───────────────┬────────────┬───────────┬────────────┐
│ policy_number ┆ location_id ┆ cobertura                       ┆ coverage     ┆ claim_number  ┆ dt_event   ┆ il_orig   ┆ year-month │
│ ---           ┆ ---         ┆ ---                             ┆ ---          ┆ ---           ┆ ---        ┆ ---       ┆ ---        │
│ i64           ┆ i64         ┆ str                             ┆ str          ┆ str           ┆ date       ┆ f64       ┆ str        │
╞═══════════════╪═════════════╪═════════════════════════════════╪══════════════╪═══════════════╪════════════╪═══════════╪════════════╡
│ 1,160,012,102 ┆ 24,791,799  ┆ Pagamento de Honorarios         ┆ OTHER        ┆ 0501619000660 ┆ 2019-08-01 ┆ 1,020.00  ┆ 201908     │
│ 1,160,012,102 ┆ 24,791,799  ┆ Danos por Água / Rompimento e … ┆ WATER DAMAGE ┆ 0501619000660 ┆ 2019-08-01 ┆ -2,000.00 ┆ 201908     │
│ 1,160,012,102 ┆ 24,791,799  ┆ RC Condom

## Adding Flag multilocation

- Our dataset has some multilocation accounts. Like 1180092388. we will count the number of locations per account. And then add this value as a new variable.

In [17]:
# adding flag multilocation
flag_multilocation = (
    raw_policy_data.group_by(pl.col("policy_number"))
    .agg(pl.col("location_id").n_unique().alias("location_id_count"))
    .with_columns(
        flag_multilocation=pl.when(pl.col("location_id_count") > 1)
        .then(pl.lit("multi location"))
        .otherwise(pl.lit("single location"))
    )
)
#
show_polars(flag_multilocation)
show_polars(
    flag_multilocation.group_by("flag_multilocation").agg(
        pl.col("policy_number").n_unique()
    )
)

raw_policy_data = raw_policy_data.join(
    flag_multilocation, on="policy_number", how="left"
)

show_polars(raw_policy_data)

shape: (130_192, 3)
┌───────────────┬───────────────────┬────────────────────┐
│ policy_number ┆ location_id_count ┆ flag_multilocation │
│ ---           ┆ ---               ┆ ---                │
│ i64           ┆ u32               ┆ str                │
╞═══════════════╪═══════════════════╪════════════════════╡
│ 1,180,095,377 ┆ 1                 ┆ single location    │
│ 1,160,024,402 ┆ 1                 ┆ single location    │
│ 1,180,093,790 ┆ 1                 ┆ single location    │
│ 1,180,031,008 ┆ 1                 ┆ single location    │
│ 1,180,035,206 ┆ 1                 ┆ single location    │
│ …             ┆ …                 ┆ …                  │
│ 1,180,025,345 ┆ 1                 ┆ single location    │
│ 1,180,058,375 ┆ 1                 ┆ single location    │
│ 1,990,015,365 ┆ 3                 ┆ multi location     │
│ 1,180,047,770 ┆ 1                 ┆ single location    │
│ 1,180,024,306 ┆ 1                 ┆ single location    │
└───────────────┴───────────────────

## Cobertura en la base de siniestralidad

In [18]:
value_counts_df(final_claims, ["cobertura", "coverage"])

shape: (64, 2)
┌─────────────────────────────────┬────────────┐
│ cobertura                       ┆ proportion │
│ ---                             ┆ ---        │
│ str                             ┆ f64        │
╞═════════════════════════════════╪════════════╡
│ Pagamento de Honorarios         ┆ 0.269      │
│ Danos Elétricos                 ┆ 0.177      │
│ Honorários de Peritos           ┆ 0.165      │
│ RC Condomínio                   ┆ 0.053      │
│ Quebra de Vidros                ┆ 0.046      │
│ Vendaval - sem Bens ao Ar livr… ┆ 0.041      │
│ Roubo de Bens                   ┆ 0.041      │
│ RC Portões                      ┆ 0.036      │
│ Danos por Água / Rompimento e … ┆ 0.035      │
│ Vendaval                        ┆ 0.032      │
│ RC Garagista                    ┆ 0.015      │
│ Incêndio                        ┆ 0.011      │
│ Básica                          ┆ 0.009      │
│ unknown                         ┆ 0.008      │
│ Roubo de Bens do Condomínio     ┆ 0.006      │
│ RC 

In [19]:
# counting unique occurrence
value_count_multilocation = raw_policy_data.select(
    pl.col("flag_multilocation")
    .fill_null(pl.lit("unknown"))
    .value_counts(normalize=True)
).unnest("flag_multilocation")

# graphing in a bar plot
multilocation_graph = value_count_multilocation.hvplot.bar(
    x=f"flag_multilocation",
    y="proportion",
    c="proportion",
    legend="right",
    width=600,
    grid=True,
    cmap="Blues",
    rot=45,
)

# creating a table with the values
multilocation_table = value_count_multilocation.hvplot.table(
    columns=["flag_multilocation", "proportion"], sortable=True, selectable=True
)

# showing both graphs
display(multilocation_graph + multilocation_table)

:Layout
   .Bars.I  :Bars   [flag_multilocation]   (proportion)
   .Table.I :Table   [flag_multilocation,proportion]

## Opening of the policies

Later in our analysis will be important a control variable Year-period. The idea is to be able to identify seasonality related with the data. This control variable also will help us to absorb some noise in the application of the GLM models. We develop a function `opening_by_period`function which allows to get done this task.

## Descriptives of inception and expiration dates

In [20]:
show_polars(
    raw_policy_data.select(pl.col("inicio_vigencia", "fin_vigencia")).describe()
)

shape: (9, 3)
┌────────────┬────────────────────────────┬────────────────────────────┐
│ statistic  ┆ inicio_vigencia            ┆ fin_vigencia               │
│ ---        ┆ ---                        ┆ ---                        │
│ str        ┆ str                        ┆ str                        │
╞════════════╪════════════════════════════╪════════════════════════════╡
│ count      ┆ 1480257                    ┆ 1480257                    │
│ null_count ┆ 0                          ┆ 0                          │
│ mean       ┆ 2021-12-19 02:46:41.690000 ┆ 2022-12-18 15:28:55.368000 │
│ std        ┆ null                       ┆ null                       │
│ min        ┆ 2019-01-01                 ┆ 2019-02-07                 │
│ 25%        ┆ 2020-08-13                 ┆ 2021-08-12                 │
│ 50%        ┆ 2022-02-04                 ┆ 2023-02-04                 │
│ 75%        ┆ 2023-04-19                 ┆ 2024-04-18                 │
│ max        ┆ 2024-08-31            

In [21]:
raw_policy_data.select(pl.col("inicio_vigencia", "fin_vigencia")).hvplot.hist(
    y=["inicio_vigencia", "fin_vigencia"],
    grid=True,
    alpha=0.5,
    width=1000,
    legend="right",
    bin_range=(date(2019, 1, 1), date(2026, 1, 3)),
    bins=7,
    title="Inception vs expiration distributions",
)

:NdOverlay   [Element]
   :Histogram   [fin_vigencia]   (fin_vigencia_count)

## Opening by period

In [22]:
# defining the opening_by period function
def opening_by_period(
    df,
    inicio_vigencia_col,
    fin_vigencia_col,
    initial_date="2019-01-01",
    cut_date="2023-12-31",
    interval="1mo",
):
    from datetime import date

    inicio_vigencia = pl.col(inicio_vigencia_col)
    fin_vigencia = pl.col(fin_vigencia_col)

    # cut_day and initial date
    cut_date_list = cut_date.split(sep="-")
    cut_day = pl.Series(
        [date(int(cut_date_list[0]), int(cut_date_list[1]), int(cut_date_list[2]))]
    )
    initial_date_list = initial_date.split(sep="-")
    initial_date = pl.Series(
        [
            date(
                int(initial_date_list[0]),
                int(initial_date_list[1]),
                int(initial_date_list[2]),
            )
        ]
    )

    df = (
        df.lazy()
        .with_columns(
            # We will start with the calculation of the duration of the policy
            total_exposure=fin_vigencia - inicio_vigencia,
            # we add a column with the desire date ranges
            date_range=pl.date_ranges(
                inicio_vigencia_col, fin_vigencia_col, interval=interval
            ),
        )
        # opening to create an entry per period
        .explode("date_range")
        # if the end of the policy is greater than the cut day, we change it
        .with_columns(
            pl.when(fin_vigencia > cut_day)
            .then(cut_day)
            .otherwise(fin_vigencia)
            .alias(fin_vigencia_col)
        )
        # if the inception of the policy is older than the initial date, we change it
        .with_columns(
            pl.when(inicio_vigencia < initial_date)
            .then(initial_date)
            .otherwise(inicio_vigencia)
            .alias(inicio_vigencia_col)
        )
        # calculating the end of the period
        .with_columns(end_period=pl.col("date_range").dt.month_end())
        # this is for the last period
        .with_columns(
            end_period=pl.when(pl.col("end_period") > fin_vigencia)
            .then(fin_vigencia)
            .otherwise(pl.col("end_period"))
        )
        .with_columns(days=pl.col("end_period") - inicio_vigencia)
        .with_columns(days_acum=pl.col("days").diff())
        # this is to return the duration of the first period
        .with_columns(pl.col("days_acum").fill_null(pl.col("days")))
        .with_columns(
            pl.when(pl.col("date_range") == inicio_vigencia)
            .then(pl.col("days"))
            .otherwise(pl.col("days_acum"))
            .alias("days_acum")
        )
        # Calculation of the final exposure
        .with_columns(exposition=pl.col("days_acum") / pl.col("total_exposure"))
        # adding the year-month variable
        .with_columns(
            year=pl.col("date_range").dt.year(),
            month=pl.col("date_range").dt.month(),
            quarter=pl.col("date_range").dt.quarter(),
        )
        .collect()
    )

    return df


openend_info = opening_by_period(
    raw_policy_data,
    "inicio_vigencia",
    "fin_vigencia",
    initial_date="2019-1-1",
    cut_date="2023-12-31",
    interval="1mo",
)
show_polars(openend_info)

shape: (19_196_268, 44)
┌───────────────┬─────────────┬────────────────────────┬───────────────────┬─────────────────┬──────────────┬──────────────┬────────────────┬─────────┬────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬─────────────┬─────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┬────────────────────────┬───────────────────┬────────────────────┬────────────────┬────────────┬────────────┬──────────────┬──────────────┬────────────┬───────┬───────┬─────────┐
│ policy_number ┆ location_id ┆ cobertura              ┆ coverage          ┆ inicio_vigencia ┆ fin_vigencia ┆ TIPO_EMISION ┆ PRODUCT_NAME   ┆ RUBRICA ┆ INDUSTRY               ┆ OCCUPANCE                       ┆ MACRO_RE

## Earned premium

In [23]:
openend_info = openend_info.with_columns(
    earned_premium=pl.col("premium_orig") * pl.col("exposition")
)
show_polars(openend_info)

shape: (19_196_268, 45)
┌───────────────┬─────────────┬────────────────────────┬───────────────────┬─────────────────┬──────────────┬──────────────┬────────────────┬─────────┬────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬─────────────┬─────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┬────────────────────────┬───────────────────┬────────────────────┬────────────────┬────────────┬────────────┬──────────────┬──────────────┬────────────┬───────┬───────┬─────────┬────────────────┐
│ policy_number ┆ location_id ┆ cobertura              ┆ coverage          ┆ inicio_vigencia ┆ fin_vigencia ┆ TIPO_EMISION ┆ PRODUCT_NAME   ┆ RUBRICA ┆ INDUSTRY               ┆ OCCUPANCE                

## Grouping by quarter

In [24]:
quarter_policies = openend_info.group_by(
    pl.all().exclude(
        "date_range",
        "end_period",
        "days",
        "days_acum",
        "exposition",
        "month",
        "earned_premium",
    )
).agg(pl.col("days_acum", "exposition", "earned_premium").sum())
show_polars(quarter_policies)

shape: (7_386_316, 41)
┌───────────────┬─────────────┬─────────────────────────────────┬───────────┬─────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬───────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┬─────────────────────────────────┬───────────────────┬────────────────────┬────────────────┬───────┬─────────┬──────────────┬────────────┬────────────────┐
│ policy_number ┆ location_id ┆ cobertura                       ┆ coverage  ┆ inicio_vigencia ┆ fin_vigencia ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY                        ┆ OCCUPANCE             

## Checking that the opening was done properly

In [25]:
# we have a sample of policies per year

sample_policies_per_year = {
    2019: 1180021073,
    2020: 1180036590,
    2021: 1180048984,
    2022: 1180060894,
    2023: 1180095272,
    2024: 1180102101,
    2024: 1180073239,
}

for year, policy in sample_policies_per_year.items():
    print(str(year) + "\n")
    policy_id = pl.col("policy_number")
    show_polars(
        quarter_policies.filter(
            (policy_id == policy) & (pl.col("cobertura") == "Incêndio")
        )
        .select(
            pl.col(
                "policy_number",
                "cobertura",
                "location_id",
                "location_id_count",
                "inicio_vigencia",
                "fin_vigencia",
                "total_exposure",
                # "date_range",
                # "end_period",
                # "days",
                "days_acum",
                "exposition",
                "year",
                "quarter",
            )
        )
        .sort(by=["location_id", "year", "quarter"]),
        all_rows=True,
    )
    print("*" * 30)

# h.filter((pl.col("inicio_vigencia") == date(2020,3,16)) & (pl.col("FLAG_MULTILOCATION")== "SINGLE LOCATION") & (pl.col("cobertura") =="Incêndio"))

2019

shape: (5, 11)
┌───────────────┬───────────┬─────────────┬───────────────────┬─────────────────┬──────────────┬────────────────┬──────────────┬────────────┬───────┬─────────┐
│ policy_number ┆ cobertura ┆ location_id ┆ location_id_count ┆ inicio_vigencia ┆ fin_vigencia ┆ total_exposure ┆ days_acum    ┆ exposition ┆ year  ┆ quarter │
│ ---           ┆ ---       ┆ ---         ┆ ---               ┆ ---             ┆ ---          ┆ ---            ┆ ---          ┆ ---        ┆ ---   ┆ ---     │
│ i64           ┆ str       ┆ i64         ┆ u32               ┆ date            ┆ date         ┆ duration[ms]   ┆ duration[ms] ┆ f64        ┆ i32   ┆ i8      │
╞═══════════════╪═══════════╪═════════════╪═══════════════════╪═════════════════╪══════════════╪════════════════╪══════════════╪════════════╪═══════╪═════════╡
│ 1,180,021,073 ┆ Incêndio  ┆ 24,909,264  ┆ 1                 ┆ 2019-01-01      ┆ 2020-01-01   ┆ 365d           ┆ 89d          ┆ 0.24       ┆ 2,019 ┆ 1       │
│ 1,180,021,073 ┆ I

## Checking which values are not matching

In [26]:
left_join = final_claims.join(
    quarter_policies, on=["policy_number", "cobertura", "year", "quarter"], how="left"
)
show_polars(left_join)

# show_polars(left_join.null_count())

filter_by_null = left_join.filter(pl.col("exposition").is_null())
filter_by_not_null = left_join.filter(pl.col("exposition").is_not_null())

show_polars(
    filter_by_null.group_by(["cobertura", "year"])
    .agg(pl.col("ultimate_il_orig").sum(), pl.col("claim_number").n_unique())
    .sort(by="ultimate_il_orig", descending=True),
    all_rows=True,
)

show_polars(
    filter_by_not_null.group_by(["cobertura", "year"])
    .agg(pl.col("ultimate_il_orig").sum(), pl.col("claim_number").n_unique())
    .sort(by="ultimate_il_orig", descending=True),
    all_rows=True,
)

# value_counts_df(filter_by_null  , ["cobertura"])

shape: (46_070, 47)
┌───────────────┬───────────────┬──────────────────────────────┬───────────────────┬───────┬────────┬──────────────────┬───────┬───────┬─────────┬─────────────┬───────────────────┬─────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬───────────────────────┬─────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┬─────────────────────────────────┬───────────────────┬────────────────────┬────────────────┬──────────────┬────────────┬────────────────┐
│ policy_number ┆ claim_number  ┆ cobertura                    ┆ coverage          ┆ line  ┆ CC_DFM ┆ ultimate_il_orig ┆ year  ┆ month ┆ quarter ┆ lo

## Final Join

In [35]:
final_df = quarter_policies.join(
    final_claims,
    on=["policy_number", "cobertura", "coverage", "year", "quarter"],
    how="left",
)

# Fixing claim_number to be numeric

final_df = final_df.with_columns(
    claim_number=pl.when(pl.col("claim_number").is_not_null())
    .then(pl.lit(1))
    .otherwise(pl.lit(0))
)
value_counts_df(final_df, ["claim_number"])
show_polars(final_df.group_by("claim_number").agg(pl.col("policy_number").n_unique()))
show_polars(final_df)

shape: (2, 2)
┌──────────────┬────────────┐
│ claim_number ┆ proportion │
│ ---          ┆ ---        │
│ i32          ┆ f64        │
╞══════════════╪════════════╡
│ 0            ┆ 0.996      │
│ 1            ┆ 0.004      │
└──────────────┴────────────┘
shape: (2, 2)
┌──────────────┬───────────────┐
│ claim_number ┆ policy_number │
│ ---          ┆ ---           │
│ i32          ┆ u32           │
╞══════════════╪═══════════════╡
│ 0            ┆ 130,192       │
│ 1            ┆ 12,647        │
└──────────────┴───────────────┘
shape: (7_389_002, 46)
┌───────────────┬─────────────┬─────────────────────────────────┬───────────┬─────────────────┬──────────────┬──────────────┬─────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬────────

## Example of the final Join

In [36]:
show_polars(
    final_df.filter(
        (pl.col("policy_number") == 1_160_040_964)
        & (pl.col("cobertura") == "RC Portões")
    ),
    all_rows=True,
)

shape: (5, 46)
┌───────────────┬─────────────┬────────────┬──────────┬─────────────────┬──────────────┬──────────────┬────────────────┬─────────┬────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬─────────────┬─────────────────────────────┬─────────────────────────────────┬─────────────────────┬──────────────┬─────────────────────────────────┬───────────────────┬────────────────────┬────────────────┬───────┬─────────┬──────────────┬────────────┬────────────────┬──────────────┬──────┬────────┬──────────────────┬───────┐
│ policy_number ┆ location_id ┆ cobertura  ┆ coverage ┆ inicio_vigencia ┆ fin_vigencia ┆ TIPO_EMISION ┆ PRODUCT_NAME   ┆ RUBRICA ┆ INDUSTRY               ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintore

## final group_by

In [37]:
final_dataframe = final_df.group_by(
    pl.all().exclude(
        "policy_number",
        "days_acum",
        "location_id",
        "inicio_vigencia",
        "fin_vigencia",
        "total_exposure",
        "exposition",
        "claim_number",
        "premium_orig",
        "total_insured_value",
        "earned_premium",
        "line",
        "ultimate_il_orig",
        "CC_DFM",
        "month",
    )
).agg(
    pl.col("exposition").sum(),
    pl.col("claim_number").sum().alias("claim_count"),
    pl.col("location_id_count").sum().alias("number_locations"),
    pl.col("total_insured_value").sum(),
    pl.col("earned_premium").sum().alias("premium_orig"),
    pl.col("ultimate_il_orig").sum(),
)

show_polars(final_dataframe)

shape: (5_036_369, 37)
┌─────────────────────────────────┬───────────────────┬──────────────┬─────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬──────────┬────────────┬────────────┬───────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬───────────────────┬────────────────────┬───────┬─────────┬────────────┬─────────────┬──────────────────┬─────────────────────┬──────────────┬──────────────────┐
│ cobertura                       ┆ coverage          ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY                        ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM2

## Example Electrical Damage

In [38]:
show_polars(final_dataframe.filter(pl.col("cobertura") == "Danos Elétricos"))

shape: (472_969, 37)
┌─────────────────┬───────────────────┬──────────────┬─────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────┬──────────────┬────────────┬───────────┬────────────┬───────────────────────────────┬──────────────┬─────────────────────┬──────────────┬────────────────────────────┬─────────────────────┬────────────┬───────┬──────┬───────────────┬────────────┬────────────┬───────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬───────────────────┬────────────────────┬───────┬─────────┬────────────┬─────────────┬──────────────────┬─────────────────────┬──────────────┬──────────────────┐
│ cobertura       ┆ coverage          ┆ TIPO_EMISION ┆ PRODUCT_NAME    ┆ RUBRICA ┆ INDUSTRY                        ┆ OCCUPANCE                       ┆ MACRO_REGION ┆ Extintores ┆ Hidrantes ┆ Sprinklers ┆ Detectores_de_Fumaça_ou_Calor ┆ CO2_ou_FM200 ┆ Brigada_de_Incendio ┆ Al

## Storing the final dataframe

In [39]:
path = r"C:\Users\abguerr\OneDrive - Chubb\2024\Ratemaking-Brazil\Archivo\parquet_data"
name = "final_dataframe_v1"
final_dataframe.write_parquet(file=path + f"\\{name}.parquet")